# CryptoPunks GPT-2 Generate

by [Khalil Meftah](https://github.com/7khalil)

*Last updated: March 23th, 2021*

This notebook will let you generate CryptoPunks sprites images with either a pre-trained model, or a model you trained with the [CryptoPunks GPT-2 Train](https://colab.research.google.com/drive/1neFtD1y3RyPdZzfY7nRwzCpMcytHdNcw#scrollTo=WN_KwM46GJFI) notebook.

Generated CryptoPunks with this notebook:
#### ![Generated CryptoPunks](https://drive.google.com/uc?id=1n6Tb5EI3F0xGg7WqrnvNgt3JOhoBzFpo)


To use this notebook just follow along with the instructions and run each cell as directed.

---

GPT-2 portions of this notebook are based on the notebook for [GPT-2-Simple](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce). The brilliant idea of using Language based model on images is from these two great notebooks [AI Generated Pokemon Sprites with GPT-2](https://colab.research.google.com/drive/1qgt8cSwKF957PgPTKhRcNwDIfWrMhbV9#scrollTo=nHZe5D9mUW7o) and [This](https://colab.research.google.com/drive/1c1kmO9tixviyBB7IGh-jVpLvOh2RpLYk).

## 1. Mount Google Drive

Google Drive is used to load pre-trained model and store the resulting generated images. Here we mount your Google Drive:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Setup

First we import everything we need:

In [2]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2

from PIL import Image
import os
import time
import re
import shutil
import pickle

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## 3. Download or Import Model

If you want to download a pre-trained model to generate CryptoPunks sprites images, run cell 3a.

If you want to import your own model that you trained with the [CryptoPunks GPT2 Train](https://colab.research.google.com/drive/1neFtD1y3RyPdZzfY7nRwzCpMcytHdNcw#scrollTo=WN_KwM46GJFI) notebook, use section 3b.

## 3a. Download a Pre-Trained Model

If you want to use a pre-trained model to generate Pokemon sprites, run this cell:

In [3]:
# @Download a Pre-Trained Model

%cd /content
!gdown -O "cryptopunks_3_run.zip" --id 1Vwj50ftdTLkoyps4KT-8-ihCLHuTOXK1
!mkdir -p checkpoint
!unzip "cryptopunks_3_run.zip" -d checkpoint

run_name = 'cryptopunks_3_run'

/content
Downloading...
From: https://drive.google.com/uc?id=1Vwj50ftdTLkoyps4KT-8-ihCLHuTOXK1
To: /content/cryptopunks_3_run.zip
100% 463M/463M [00:03<00:00, 130MB/s]
Archive:  cryptopunks_3_run.zip
   creating: checkpoint/cryptopunks_3_run/
  inflating: checkpoint/cryptopunks_3_run/encoder.json  
  inflating: checkpoint/cryptopunks_3_run/checkpoint  
 extracting: checkpoint/cryptopunks_3_run/counter  
  inflating: checkpoint/cryptopunks_3_run/hparams.json  
  inflating: checkpoint/cryptopunks_3_run/model-3000.index  
  inflating: checkpoint/cryptopunks_3_run/vocab.bpe  
  inflating: checkpoint/cryptopunks_3_run/model-3000.meta  
  inflating: checkpoint/cryptopunks_3_run/events.out.tfevents.1616368435.084155c5f26d  
  inflating: checkpoint/cryptopunks_3_run/model-3000.data-00000-of-00001  


## 4. Generation

Here we'll generate your punks sprites images with the model we imported and save them to Google Drive.

There are a few settings you can change for this cell:

- output_folder - This is the folder that sprites will be saved to in your Google Drive
- generate_count - This is how many sprites will be generated when you run this cell
- temperature - This controls how "crazy" the output is. A high number will lead weird looking cryptpunk, but might be interesting. Too low will be boring. Play with this number.




In [4]:
# @Generation settings

output_folder = 'output_punks' #@param {type:"string"}
generate_count = 1000 #@param {type:"integer"}
temperature = 0.7 #@param {type:"slider", min:0.7, max:1.3, step:0.01}

## 4a. Create utils functions

First, we import the decoder file.
Then, we create some helper functions to convert the text back to RGB images.

Run this cell if you haven't trained your own model.

In [5]:
!gdown -O "decode.pkl" --id 1pIEIPPaDToVSDEOmzsVIXdygPrKcBfEE
with open('decode.pkl', 'rb') as handle:
    decode_map = pickle.load(handle)

Downloading...
From: https://drive.google.com/uc?id=1pIEIPPaDToVSDEOmzsVIXdygPrKcBfEE
To: /content/decode.pkl
100% 5.01k/5.01k [00:00<00:00, 4.84MB/s]


Run this cell to create the helper functions.

In [6]:
def createImage(text, path):
    lines = text.split('\n')

    imageWidth = 24
    imageHeight = 24

    pixels = []
    for y in range(imageHeight):
        for x in range(imageWidth):
            pixels.append((100, 132, 150, 255))

    for line in lines:
        split = line.split(' ')

        marker = split[0]
        match = re.search("^([0-9]+)([d])$", marker)

        if match:
            groups = match.groups()
            index = int(groups[0])

            for x in range(len(split) - 1):
                s = split[x + 1]
                match = re.search("^[0-9][0-9][0-9]$", s)
                if not match:
                    s = '079'

                rgb = eval(decode_map[s])

                i = (index * imageWidth) + x
                pixels[i] = (rgb[0], rgb[1], rgb[2], 255)

    image = Image.new('RGBA', (imageWidth, imageHeight))
    image.putdata(pixels)
    image.save(path)


def blankLines():
    lines = []

    for i in range(0, height):
        lines.append('')

    return lines


sess = None

## 4b. Start generating Punks

Finally, we can run this cell to generate CryptoPunks images imagined by AI.

In [ ]:
if not os.path.exists('/content/drive/MyDrive/%s' % output_folder):
    os.makedirs('/content/drive/MyDrive/%s' % output_folder)


prefix = """
00d 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079
01d 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079
02d 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079
03d 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079
04d 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079 079
"""

for img in range(0, generate_count):
    print("punk number: %i" % img)

    if not sess:
        sess = gpt2.start_tf_sess()
    else:
        sess = gpt2.reset_session(sess)
    
    gpt2.load_gpt2(sess, run_name=run_name)
    width = 24
    height = 24
    
    text = gpt2.generate(sess, run_name=run_name, prefix=prefix, temperature=temperature, return_as_list=True)[0]
    print("\nGPT-2 text output: \n")
    print(text)

    lines = text.split('\n')
    section = '\n'.join(lines[20:])
    newText = gpt2.generate(sess, run_name=run_name, prefix=section, temperature=temperature, return_as_list=True)[0]
    newLines = newText.split('\n')
    lines = lines[:20] + newLines
    splits = [line.split(' ') for line in lines]

    indices = [split[0] for split in splits]
    idx = indices.index('00d')

    for i, split in enumerate(splits):
        if len(split) < width + 1:
            while len(split) < width + 1:
                split.append('079')

            lines[i] = ' '.join(split)
        else:
            split = split[:width+1]
            lines[i] = ' '.join(split)
    
    output = '\n'.join(lines[:height])
    print("\nSaving output: \n")
    print(output)
    filename = '%i' % int(time.time())
    createImage(output, '/content/drive/MyDrive/%s/%s.png' % (output_folder, filename))
    print('\nsaved !\n')

Streaming output truncated to the last 5000 lines.
08d 079 079 079 079 079 079 220 220 220 220 220 220 220 220 220 220 220 220 079 079 079 079 079 079
09d 079 079 079 079 079 079 220 220 220 220 220 220 220 220 220 220 220 220 079 079 079 079 079 079
10d 079 079 079 079 079 079 220 220 220 096 096 096 096 220 096 096 220 079 079 079 079 079 079 079
11d 079 079 079 079 079 079 220 220 096 014 096 096 096 096 014 096 220 079 079 079 079 079 079 079
12d 079 079 079 079 079 220 220 096 096 019 019 096 096 096 019 019 220 079 079 079 079 079 079 079
13d 079 079 079 079 079 220 220 096 096 000 016 096 096 096 000 016 220 079 079 079 079 079 079 079
14d 079 079 079 079 079 220 220 220 096 014 096 096 096 096 014 096 220 079 079 079 079 079 079 079
15d 079 079 079 079 079 220 220 220 096 096 096 096 096 096 096 096 220 079 079 079 079 079 079 079
16d 079 079 079 079 079 220 220 220 096 096 096 096 000 096 096 096 220 079 079 079 079 079 079 079
17d 079 079 079 079 079 220 220 220 096 096 096 0

## 5. Done !

You're done ! Check out the generated CryptoPunks images in your chosen Google Drive folder. Run step 4b again to generate more. Cool !